In [478]:
#SPOTIFY FEATURES
import requests
import pandas as pd
import time
import os


In [479]:
#Credentials - Oauth to access track features
client_id =  '5ce71a6138664aff9b24f6da2da63333'
secret = '166aca7c7cce4b168cd95f6f7429bc98'

Oauth ='BQBVC84OMxnxTc2XjrmzBSljKnBNTrNw3Z0G-6qKDRwctdTGxX1gpbouxwCOL7FO5oBkzL0nB8GcgSjKAKK3wLD9lrRdFbcOeDbKsm4o8jOvcMFAfqz9zqsPPd2aGKR66xO6d_juea4e'


In [437]:
#Splitting unique songs into two lists because limit per request is 100

first = unique[:100]
second = unique[100:]

# creates comma separated string of tracks to pass into API calls
i = ''
for x in first:
    if x == first[-1]:
        i = i+x
    else:
        i = i+x+','
j = ''
for x in second:
    if x == second[-1]:
        j = j+x
    else:
        j = j+x+','
both = [i,j]

result = []
for x in both:
    track_ids = x
    result = []
    def spotify_call(url_params,Oauth,track_ids):

        url= 'https://api.spotify.com/v1/audio-features'
        headers = {"Authorization": "Bearer {}".format(Oauth),}
        url_params = {'ids' : track_ids.replace(' ', '+'),}

        response = requests.get(url,headers=headers,params= url_params)
        result.append(response.json())

In [457]:
#API call to Spotify to get track features of songs

def spotify_call(both):
    result = []
    for x in both:
        track_ids = x
        url= 'https://api.spotify.com/v1/audio-features'
        headers = {"Authorization": "Bearer {}".format(Oauth),}
        url_params = {'ids' : track_ids.replace(' ', '+'),}
        response = requests.get(url,headers=headers,params= url_params)
        result.append(response.json()) 
    return result


In [458]:
a = spotify_call(both)

In [475]:
df1 = pd.DataFrame(a[0]['audio_features'])
df2 = pd.DataFrame(a[1]['audio_features'])
df = pd.concat([df1,df2])

In [477]:
df

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.48500,https://api.spotify.com/v1/audio-analysis/7KPG...,0.783,159152,0.692,7KPGeiXWDsGEvFK62wzU8E,0.000000,10,0.1200,-6.156,0,0.3290,100.082,4,https://api.spotify.com/v1/tracks/7KPGeiXWDsGE...,audio_features,spotify:track:7KPGeiXWDsGEvFK62wzU8E,0.260
1,0.05670,https://api.spotify.com/v1/audio-analysis/38Eq...,0.803,167707,0.533,38Eq6RgIxOdt4twqyVAPCn,0.032300,7,0.2810,-7.331,1,0.0656,141.996,4,https://api.spotify.com/v1/tracks/38Eq6RgIxOdt...,audio_features,spotify:track:38Eq6RgIxOdt4twqyVAPCn,0.255
2,0.10200,https://api.spotify.com/v1/audio-analysis/4MSF...,0.735,174507,0.764,4MSFxqmkiMxq7e61jTqgqt,0.000000,4,0.2450,-4.626,0,0.1550,97.056,4,https://api.spotify.com/v1/tracks/4MSFxqmkiMxq...,audio_features,spotify:track:4MSFxqmkiMxq7e61jTqgqt,0.609
3,0.44700,https://api.spotify.com/v1/audio-analysis/5f98...,0.631,343446,0.381,5f9808hpiCpuNyqqdXmpF2,0.000171,7,0.0902,-11.808,1,0.0400,126.008,4,https://api.spotify.com/v1/tracks/5f9808hpiCpu...,audio_features,spotify:track:5f9808hpiCpuNyqqdXmpF2,0.360
4,0.06370,https://api.spotify.com/v1/audio-analysis/7to6...,0.590,253077,0.562,7to68V64Cu6zk0UDo5tyw3,0.000000,11,0.1050,-9.607,0,0.0505,140.835,4,https://api.spotify.com/v1/tracks/7to68V64Cu6z...,audio_features,spotify:track:7to68V64Cu6zk0UDo5tyw3,0.561
5,0.50500,https://api.spotify.com/v1/audio-analysis/1IPR...,0.642,160357,0.598,1IPRghUN3LFPNbCTsfBJx9,0.000000,10,0.1290,-8.716,0,0.2830,170.185,4,https://api.spotify.com/v1/tracks/1IPRghUN3LFP...,audio_features,spotify:track:1IPRghUN3LFPNbCTsfBJx9,0.508
6,0.02010,https://api.spotify.com/v1/audio-analysis/2X8s...,0.681,163851,0.940,2X8sssaxS5GReg6945B6RW,0.000000,0,0.5780,-1.770,1,0.0386,124.015,4,https://api.spotify.com/v1/tracks/2X8sssaxS5GR...,audio_features,spotify:track:2X8sssaxS5GReg6945B6RW,0.648
7,0.01650,https://api.spotify.com/v1/audio-analysis/2D3z...,0.747,158800,0.740,2D3z17LBMJ2HEHeBFFjTLi,0.000099,4,0.1010,-4.325,0,0.1910,176.059,4,https://api.spotify.com/v1/tracks/2D3z17LBMJ2H...,audio_features,spotify:track:2D3z17LBMJ2HEHeBFFjTLi,0.552
8,0.15500,https://api.spotify.com/v1/audio-analysis/4NkK...,0.828,202528,0.855,4NkKfZe4USK76vt2dVnIQ8,0.000000,7,0.2860,-5.314,1,0.1350,128.056,4,https://api.spotify.com/v1/tracks/4NkKfZe4USK7...,audio_features,spotify:track:4NkKfZe4USK76vt2dVnIQ8,0.261
9,0.29700,https://api.spotify.com/v1/audio-analysis/7oxS...,0.685,216326,0.761,7oxSh7uiB74lWGVIv2Qtlm,0.000000,11,0.2340,-4.556,0,0.0641,159.929,4,https://api.spotify.com/v1/tracks/7oxSh7uiB74l...,audio_features,spotify:track:7oxSh7uiB74lWGVIv2Qtlm,0.726


In [302]:
# List of country codes on spotifycharts.com - Removed ad & mc because they had no data for last week

country_codes = ['ar', 'at', 'au', 'be', 'bg', 'bo', 'br', 'ca', 'ch', 'cl', 'co', 'cr',
                 'cy', 'cz', 'de', 'dk', 'do', 'ec', 'ee', 'es', 'fi', 'fr', 'gb', 'gr', 'gt',
                 'hk', 'hn', 'hu', 'id', 'ie', 'il', 'is', 'it', 'jp', 'lt', 'lu', 'lv',
                 'mt', 'mx','my', 'ni', 'nl', 'no', 'nz', 'pa', 'pe', 'ph', 'pl', 'pt', 'py',
                 'ro', 'se', 'sg', 'sk', 'sv', 'th', 'tr', 'tw', 'us', 'uy', 'vn']

#Webscraping 

from bs4 import BeautifulSoup

#Loop to scrape top 10 songs (out of 200) by country and add Ids to a list for each country
top10_bycountry = {}
for country in country_codes:
    charts = f'https://spotifycharts.com/regional/{country}/weekly/2019-01-25--2019-02-01'
    html_page = requests.get(charts) #Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    top_links = []
    top10_links = []
    for link in soup.findAll('a',target="_blank"):
        top_links.append((link.get('href')))
        top10_links = top_links[1:11]

    top10_ids = []
    for x in top10_links:
        x = x[-22:]
        top10_ids.append(x)
    top10_bycountry[country] = top10_ids

In [299]:
#Checking which country returns empty ID for the week
counter = 0
for k,v in top10_bycountry.items():
    if len(v) == 0:
        print(k)
        counter +=1
counter

In [311]:
len(country_codes)

61

In [419]:
all_songs = []
for n in list(top10_bycountry.values()):
    for x in n:
        all_songs.append(x)
unique = list(set(all_songs))

In [301]:
country_codes_all = ['ad', 'ar', 'at', 'au', 'be', 'bg', 'bo', 'br', 'ca', 'ch', 'cl', 'co', 'cr',
                 'cy', 'cz', 'de', 'dk', 'do', 'ec', 'ee', 'es', 'fi', 'fr', 'gb', 'gr', 'gt',
                 'hk', 'hn', 'hu', 'id', 'ie', 'il', 'is', 'it', 'jp', 'lt', 'lu', 'lv', 'mc',
                 'mt', 'mx','my', 'ni', 'nl', 'no', 'nz', 'pa', 'pe', 'ph', 'pl', 'pt', 'py',
                 'ro', 'se', 'sg', 'sk', 'sv', 'th', 'tr', 'tw', 'us', 'uy', 'vn']

In [516]:
# Transpose column countries into row names. Columns become the ranking of top10.
top10_byct_df = (pd.DataFrame(top10_bycountry)).T
top10_byct_df

,0,1,2,3,4,5,6,7,8,9
ar,5iwz1NiezX7WWjnCgY5TH4,6FyRXC8tJUh863JCkyWqtk,0s5DNDV6LAKn2u2Ufa3VjQ,6MJUCumnQsQEKbCy28tbCP,3dLiBBKbeUIhzGNeuo5JGS,116H0KvKr2Zl4RPuVBruDO,5W83ErFkO3aKAIS1WMi6u0,2D3z17LBMJ2HEHeBFFjTLi,22sESxzbY1O1E19xjOTaYf,4w8niZpiMy6qz1mntFA5uM
at,7r55d7IhAj5UGKiYbXBwRP,260R90ctQ1SiPax90PuEsT,14msK75pk3pA33pzPVNtBF,7KPGeiXWDsGEvFK62wzU8E,25sgk305KZfyuqVBQIahim,63THFs9ovzO3WQsz4lzh0H,1rqqCSm0Qe4I9rUvWncaom,1IPRghUN3LFPNbCTsfBJx9,2PAYSgrVOBz8RJqp9B7QLu,5z3vuYzS9UF6rD6TmBEdUf
au,14msK75pk3pA33pzPVNtBF,3KkXRkHbMCARz0aVfEt68P,6MWtB6iiXyIwun0YzU6DFP,5p7ujcrUXASCNwRaWNHR1C,6Qs4SXO9dwPj5GKvVOv8Ki,25sgk305KZfyuqVBQIahim,2rPE9A1vEgShuZxxzR2tZH,2xLMifQCjDGFmkHkpNLD9h,7FGq80cy8juXBCD2nrqdWU,7to68V64Cu6zk0UDo5tyw3
be,14msK75pk3pA33pzPVNtBF,25sgk305KZfyuqVBQIahim,1rqqCSm0Qe4I9rUvWncaom,5p7ujcrUXASCNwRaWNHR1C,2VxeLyX666F8uXCJ0dZF8B,6MWtB6iiXyIwun0YzU6DFP,3EPXxR3ImUwfayaurPi3cm,2rPE9A1vEgShuZxxzR2tZH,5itOtNx0WxtJmi1TQ3RuRd,6Qs4SXO9dwPj5GKvVOv8Ki
bg,14msK75pk3pA33pzPVNtBF,5p7ujcrUXASCNwRaWNHR1C,6MWtB6iiXyIwun0YzU6DFP,2JvzF1RMd7lE3KmFlsyZD8,7wFybC8jBH3zE139OpCtpG,2xLMifQCjDGFmkHkpNLD9h,2VxeLyX666F8uXCJ0dZF8B,2IRZnDFmlqMuOrYOLnZZyc,6Qs4SXO9dwPj5GKvVOv8Ki,54ikdt0wnIMn8sT9NFgFSS
bo,0s5DNDV6LAKn2u2Ufa3VjQ,5iwz1NiezX7WWjnCgY5TH4,5W83ErFkO3aKAIS1WMi6u0,6FyRXC8tJUh863JCkyWqtk,059bcIhyc2SBwm6sw2AZzd,116H0KvKr2Zl4RPuVBruDO,3ao3OVxHlb3C08vAaApUlm,1mT47pKLXUwPYGwB91GcNz,2cYnZNzFRFK0H7vSNxPsUq,4w8niZpiMy6qz1mntFA5uM
br,5EKHbW7mXcAR4hCyS3YXjb,5ixqGGFu0hdHfErdp32QwT,2qkNnJvQvhLiZ63X1L3eFz,5yH8pYxNckOU1cxfPsMIaz,2lAyctg5FocS67eksHoyIg,1T7aSWZQIorPMzRT8jTMXc,3Hb9kUdm4yf839Fle4RIdT,5Wmg5LPPJnbMsKA9fthM3l,2X8sssaxS5GReg6945B6RW,0kCqaybZgYcLYxzZymmBoc
ca,14msK75pk3pA33pzPVNtBF,2JvzF1RMd7lE3KmFlsyZD8,6MWtB6iiXyIwun0YzU6DFP,3KkXRkHbMCARz0aVfEt68P,2IRZnDFmlqMuOrYOLnZZyc,5p7ujcrUXASCNwRaWNHR1C,2xLMifQCjDGFmkHkpNLD9h,6Qs4SXO9dwPj5GKvVOv8Ki,3Ol2xnObFdKV9pmRD2t9x8,2rPE9A1vEgShuZxxzR2tZH
ch,7r55d7IhAj5UGKiYbXBwRP,14msK75pk3pA33pzPVNtBF,260R90ctQ1SiPax90PuEsT,25sgk305KZfyuqVBQIahim,1rqqCSm0Qe4I9rUvWncaom,5p7ujcrUXASCNwRaWNHR1C,7KPGeiXWDsGEvFK62wzU8E,63THFs9ovzO3WQsz4lzh0H,6MWtB6iiXyIwun0YzU6DFP,2VxeLyX666F8uXCJ0dZF8B
cl,5iwz1NiezX7WWjnCgY5TH4,1LiN0Z98FkR1t0m8KmLcAH,0EJu1RBtxzldMG2peczuPv,5W83ErFkO3aKAIS1WMi6u0,6FyRXC8tJUh863JCkyWqtk,6MJUCumnQsQEKbCy28tbCP,116H0KvKr2Zl4RPuVBruDO,42ArfZUgGo3k5btIHp08it,0s5DNDV6LAKn2u2Ufa3VjQ,0GmiGIxXDNrePrHBpijYDp


In [544]:
stacked_df = top10_byct_df.stack()
df3 = pd.DataFrame(stacked_df.rename('id'))

id
ar 0  5iwz1NiezX7WWjnCgY5TH4
   1  6FyRXC8tJUh863JCkyWqtk
   2  0s5DNDV6LAKn2u2Ufa3VjQ
   3  6MJUCumnQsQEKbCy28tbCP
   4  3dLiBBKbeUIhzGNeuo5JGS
   5  116H0KvKr2Zl4RPuVBruDO
   6  5W83ErFkO3aKAIS1WMi6u0
   7  2D3z17LBMJ2HEHeBFFjTLi
   8  22sESxzbY1O1E19xjOTaYf
   9  4w8niZpiMy6qz1mntFA5uM
at 0  7r55d7IhAj5UGKiYbXBwRP
   1  260R90ctQ1SiPax90PuEsT
   2  14msK75pk3pA33pzPVNtBF
   3  7KPGeiXWDsGEvFK62wzU8E
   4  25sgk305KZfyuqVBQIahim
   5  63THFs9ovzO3WQsz4lzh0H
   6  1rqqCSm0Qe4I9rUvWncaom
   7  1IPRghUN3LFPNbCTsfBJx9
   8  2PAYSgrVOBz8RJqp9B7QLu
   9  5z3vuYzS9UF6rD6TmBEdUf

In [552]:
df3.reset_index(inplace=True)

In [ ]:
#df3 = df3.reset_index()

In [557]:
df3.rename(columns={'level_0':'country','level_1':'rank'},inplace=True)

,country,rank,id
0,ar,0,5iwz1NiezX7WWjnCgY5TH4
1,ar,1,6FyRXC8tJUh863JCkyWqtk
2,ar,2,0s5DNDV6LAKn2u2Ufa3VjQ
3,ar,3,6MJUCumnQsQEKbCy28tbCP
4,ar,4,3dLiBBKbeUIhzGNeuo5JGS


In [558]:
merged_top10_features = df3.merge(df,how='left')

In [559]:
merged_top10_features

,country,rank,id,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,ar,0,5iwz1NiezX7WWjnCgY5TH4,0.32300,https://api.spotify.com/v1/audio-analysis/5iwz...,0.826,238200,0.773,0.000000,11,0.1430,-4.218,0,0.0524,126.899,4,https://api.spotify.com/v1/tracks/5iwz1NiezX7W...,audio_features,spotify:track:5iwz1NiezX7WWjnCgY5TH4,0.761
1,ar,1,6FyRXC8tJUh863JCkyWqtk,0.32300,https://api.spotify.com/v1/audio-analysis/6FyR...,0.767,258639,0.709,0.000000,1,0.0676,-4.470,1,0.3360,171.993,4,https://api.spotify.com/v1/tracks/6FyRXC8tJUh8...,audio_features,spotify:track:6FyRXC8tJUh863JCkyWqtk,0.720
2,ar,2,0s5DNDV6LAKn2u2Ufa3VjQ,0.14600,https://api.spotify.com/v1/audio-analysis/0s5D...,0.709,204453,0.771,0.000003,7,0.2490,-4.598,1,0.1660,174.006,4,https://api.spotify.com/v1/tracks/0s5DNDV6LAKn...,audio_features,spotify:track:0s5DNDV6LAKn2u2Ufa3VjQ,0.709
3,ar,3,6MJUCumnQsQEKbCy28tbCP,0.29100,https://api.spotify.com/v1/audio-analysis/6MJU...,0.788,207853,0.771,0.000056,4,0.1720,-3.206,0,0.0563,99.038,4,https://api.spotify.com/v1/tracks/6MJUCumnQsQE...,audio_features,spotify:track:6MJUCumnQsQEKbCy28tbCP,0.701
4,ar,4,3dLiBBKbeUIhzGNeuo5JGS,0.42000,https://api.spotify.com/v1/audio-analysis/3dLi...,0.641,244771,0.652,0.000002,8,0.0940,-8.575,0,0.2640,175.022,4,https://api.spotify.com/v1/tracks/3dLiBBKbeUIh...,audio_features,spotify:track:3dLiBBKbeUIhzGNeuo5JGS,0.406
5,ar,5,116H0KvKr2Zl4RPuVBruDO,0.01410,https://api.spotify.com/v1/audio-analysis/116H...,0.817,210368,0.539,0.000496,6,0.0990,-6.349,0,0.0621,97.062,4,https://api.spotify.com/v1/tracks/116H0KvKr2Zl...,audio_features,spotify:track:116H0KvKr2Zl4RPuVBruDO,0.158
6,ar,6,5W83ErFkO3aKAIS1WMi6u0,0.60200,https://api.spotify.com/v1/audio-analysis/5W83...,0.807,258800,0.803,0.008530,11,0.1360,-4.156,1,0.1260,91.987,4,https://api.spotify.com/v1/tracks/5W83ErFkO3aK...,audio_features,spotify:track:5W83ErFkO3aKAIS1WMi6u0,0.706
7,ar,7,2D3z17LBMJ2HEHeBFFjTLi,0.01650,https://api.spotify.com/v1/audio-analysis/2D3z...,0.747,158800,0.740,0.000099,4,0.1010,-4.325,0,0.1910,176.059,4,https://api.spotify.com/v1/tracks/2D3z17LBMJ2H...,audio_features,spotify:track:2D3z17LBMJ2HEHeBFFjTLi,0.552
8,ar,8,22sESxzbY1O1E19xjOTaYf,0.21400,https://api.spotify.com/v1/audio-analysis/22sE...,0.531,177787,0.752,0.000000,9,0.1450,-3.330,0,0.2000,192.443,4,https://api.spotify.com/v1/tracks/22sESxzbY1O1...,audio_features,spotify:track:22sESxzbY1O1E19xjOTaYf,0.884
9,ar,9,4w8niZpiMy6qz1mntFA5uM,0.15300,https://api.spotify.com/v1/audio-analysis/4w8n...,0.841,212500,0.798,0.000003,1,0.0618,-4.206,0,0.2290,95.948,4,https://api.spotify.com/v1/tracks/4w8niZpiMy6q...,audio_features,spotify:track:4w8niZpiMy6qz1mntFA5uM,0.591


In [568]:
average_features = merged_top10_features.groupby('country').mean()

In [581]:
average_features.sort_values('energy',ascending = False)

,rank,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
country,,,,,,,,,,,,,,
br,4.5,0.399910,0.7148,166936.4,0.8130,4.540000e-07,6.4,0.44059,-3.3003,0.6,0.09571,136.9498,4.0,0.65790
bo,4.5,0.214670,0.7896,221053.8,0.7471,9.213220e-04,6.3,0.11062,-4.1762,0.7,0.13470,114.3072,4.0,0.66440
pa,4.5,0.265220,0.7691,218169.1,0.7450,9.166490e-04,7.9,0.13179,-4.0124,0.4,0.13228,133.0277,4.0,0.60130
ec,4.5,0.228470,0.7843,220589.1,0.7444,9.266090e-04,6.6,0.12164,-4.0762,0.7,0.11743,114.6162,4.0,0.67540
jp,4.5,0.148175,0.6219,249978.7,0.7428,1.140000e-06,3.9,0.11005,-4.8187,0.9,0.06754,106.1545,4.0,0.52120
uy,4.5,0.211870,0.7649,208580.0,0.7410,9.188220e-04,6.5,0.13914,-4.1608,0.3,0.15298,132.5401,4.0,0.63150
co,4.5,0.263970,0.7775,219096.2,0.7319,9.120090e-04,6.7,0.13779,-4.4086,0.6,0.13648,122.7139,4.0,0.68820
pe,4.5,0.259970,0.7847,223201.6,0.7311,9.266090e-04,6.3,0.12724,-4.3376,0.6,0.11370,114.9222,4.0,0.67300
ar,4.5,0.250260,0.7474,217217.1,0.7308,9.189960e-04,6.2,0.12684,-4.7433,0.3,0.16828,140.0457,4.0,0.61880
